iterate all files related to CAPS



In [99]:
import pandas as pd

store each into one big df


In [100]:
import os

#https://stackoverflow.com/a/21232849 model 
def getCAPSByDateAndType(type):  #returns a dict, date + df caps for that date, then extended date and time
                                # print("for type, ", type)
    ret = []
    for root, dirs, files in os.walk("."):
        for filename in files:
            if type in filename:
                # print("CAPS file, ", filename)
                ret.append(filename)
    return ret

arr = getCAPSByDateAndType("CAPS")
li = [] 
for filename in arr:
    csv = "lob_caps/" + filename
    print(csv)
    df = pd.read_csv(csv, index_col=None, header=0)
    li.append(df)

endFrame = pd.concat(li, axis=0, ignore_index=True)
endFrame.sort_values(by=['time'], ascending=True)
print("for new df: ", endFrame.shape[0])
start = endFrame["time"].min()
end = endFrame["time"].max()
print("start: ", start, " end: ", end)

lob_caps/Fri Aug 26 2022 06:27:17 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Sat Oct 01 2022 08:39:01 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Fri Dec 02 2022 09:40:57 GMT-0800 (Pacific Standard Time)-CAPS.csv
lob_caps/Thu Aug 25 2022 10:42:37 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Mon Oct 31 2022 17:15:31 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Tue Nov 08 2022 22:29:16 GMT-0800 (Pacific Standard Time)-CAPS.csv
lob_caps/Fri Nov 25 2022 07:50:10 GMT-0800 (Pacific Standard Time)-CAPS.csv
lob_caps/Thu Oct 27 2022 07:39:06 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Wed Oct 12 2022 07:36:21 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Thu Dec 01 2022 07:42:09 GMT-0800 (Pacific Standard Time)-CAPS.csv
lob_caps/Sat Oct 22 2022 08:57:53 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Tue Sep 13 2022 06:57:20 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Tue Oct 25 2022 06:50:17 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Sat

In [101]:
def getIndicators():
    return pd.read_csv("usIndicatorsFall22-fulls.csv")

In [102]:
from dateutil import parser
import datetime
import time

def convertIndicatorDateToEpoch(original):
    # print("original ", original) #Thu Aug 11 4:30 AM 2022
    re =time.strptime(original, "%a %b %d %I:%M %p %Y")
    ep = time.mktime(re)
    # print("TS ", int(ep))   #1660311116579
    return int(ep)

In [103]:
                                # print(type(inds), inds.columns)
def genDatesForIndicators():
    inds = getIndicators() 
    ret = []               
    for key, val in inds.iterrows():
                                    # print(key, val["date"], val["time"])
        s1 = val['date'].replace(','  , '')
        ss = s1.split(" ")
        day  = ss[0][0:3]
        mon = ss[1][0:3]
        num = ss[2]
        hr = val['time']
        if int(num) <10:
            num = "0"+num
        yr = ss[3]
        l = day + " " + mon + " " + num #+ " " + hr
        extendedDateTime = day + " " + mon + " " + num + " " + hr + " " +yr
                            # print("FOR INDICATOR, DATED: ",extendedDateTime)
        d = convertIndicatorDateToEpoch(extendedDateTime)
        # print(d)
        ret.append(d)
    return ret
        
dateArr = genDatesForIndicators()
for i in range(len(dateArr)):
    print(dateArr[i])

1659358800
1659445200
1659531600
1659612600
1659699000
1660131000
1660131000
1660217400
1660309200
1660649400
1660735800
1660755600
1661340600
1661513400
1661513400
1661518800
1661864400
1662037200
1662118200
1662469200
1662550200
1663155000
1663241400
1663333200
1663673400
1663779600
1663779600
1664278200
1664537400
1664537400
1664802000
1664888400
1664969400
1664974800
1665142200
1665574200
1665594000
1665660600
1665660600
1665747000
1665752400
1666179000
1666870200
1666956600
1666956600
1667307600
1667307600
1667410200
1667475000
1667561400
1668087000
1668087000
1668178800


endFrame: collected time series, several months, 111K+ events

dateArr: list of all dates, as epochs for all indicators (dates only)

In [140]:
# dateArr = 1667561400
# endFrame['time']<= dateArr

l = endFrame[(endFrame['time'] <= 1668178800 ) & (endFrame['time'] > 1664578800   )] #1664578800  -->  1668178800
print(l.shape[0])

0


In [141]:
thresh = ((1000 * 60) * 60 ) * 1
print("threshold minus gate: ", thresh)
# print(endFrame.dtypes)
#get moments 1 hours before the announcement
print(endFrame["time"].min(), " --> ", endFrame["time"].max(), "  --> ", endFrame.shape[0], " items")

for i in range(len(dateArr)): 
    # print(type(dateArr[i]))  #int 
    lowEnd = dateArr[i] - thresh  
    print(lowEnd, " --> ", dateArr[i])   #
    # sample = endFrame[endFrame['time'].between(dateArr[i], lowEnd, inclusive="both")]
    # set = endFrame.loc[
    # sample1 = endFrame[(endFrame['time']> lowEnd)]
    # sample = endFrame[(endFrame['time']<= dateArr[i])]# ] #(endFrame['time']<= dateArr[i])]    & (endFrame['time']> lowEnd)
    s =endFrame[(endFrame['time'] > lowEnd ) & (endFrame['time'] <= dateArr[i] )]
    # sample = endFrame.loc[s]
    # sample = endFrame.loc[set]
    # set = endFrame.query(endFrame['time'] >= lowEnd & endFrame['time'] <= dateArr[i]).index
    # sample = endFrame[set]
    # if int(sample.shape[0]) > 0:
    print(s.head()) #ok  
    # print(sample1.shape[0])
        # print(sample["time"].min(), " --> ", sample["time"].max(), "  --> ", sample.shape[0], " items")
    

threshold minus gate:  3600000
1660221600292  -->  1671726692823   -->  111336  items
1655758800  -->  1659358800
Empty DataFrame
Columns: [bc, ac, tbv, tav, time, mp, minBid]
Index: []
1655845200  -->  1659445200
Empty DataFrame
Columns: [bc, ac, tbv, tav, time, mp, minBid]
Index: []
1655931600  -->  1659531600
Empty DataFrame
Columns: [bc, ac, tbv, tav, time, mp, minBid]
Index: []
1656012600  -->  1659612600
Empty DataFrame
Columns: [bc, ac, tbv, tav, time, mp, minBid]
Index: []
1656099000  -->  1659699000
Empty DataFrame
Columns: [bc, ac, tbv, tav, time, mp, minBid]
Index: []
1656531000  -->  1660131000
Empty DataFrame
Columns: [bc, ac, tbv, tav, time, mp, minBid]
Index: []
1656531000  -->  1660131000
Empty DataFrame
Columns: [bc, ac, tbv, tav, time, mp, minBid]
Index: []
1656617400  -->  1660217400
Empty DataFrame
Columns: [bc, ac, tbv, tav, time, mp, minBid]
Index: []
1656709200  -->  1660309200
Empty DataFrame
Columns: [bc, ac, tbv, tav, time, mp, minBid]
Index: []
1657049400  --

iterate all anouncement times, associate 1,000 data points before then after each, then calculate